# Analyzing results

In [ ]:
import sys,os
root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from lib.utils import utils
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt

## Computing Triples

### Get a number of games

In [ ]:
start_index = 0
num_games = 100
path = "../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl"
games_jsons = utils.load_jsonl(path, num_games=num_games, mmap=False, completed_only=True)

In [ ]:
# Convert to a pandas dataframe
df = pd.DataFrame(games_jsons)

In [ ]:
games = df['phases']
games.apply(lambda x: x[0])

In [ ]:
for game in games:
    for ix, iy in enumerate(game):
        game[ix]['phase_id'] = ix

### Flatten json records

In [ ]:
all_records = []
for idx, game in enumerate(games):
    for idx, phase in enumerate(game):
        row_generator = utils.flatten_json(phase)
        assert row_generator is not None, row_generator
        for row in row_generator:
            all_records.append(row.copy())

### Get the complete dataframe with an added column

In [ ]:
complete_df = pd.DataFrame.from_records(all_records)
complete_df['unique_unit_id'] = -1

### Get the unique games in the dataframe

In [ ]:
unique_games = complete_df["game_id"].unique()

### Do temporal binding of units

In [ ]:
game_phase_df_list = []
for idx, game_id in enumerate(unique_games):
    phases_df_list = []
    print(idx, game_id, flush=True)
    s_dict = {}
    d_dict = {}
    _id = 1
    game_df = complete_df.loc[complete_df["game_id"].apply(lambda x: x == game_id)]
    unique_phases = game_df['phase_id'].unique()
    for phase in unique_phases:
        condition = game_df["phase_id"].apply(lambda x: x == phase)
        phase_df = game_df.loc[condition]
        s_dict, d_dict, _id = utils.assign_unit_id(phase_df, s_dict, d_dict, _id, discard_disband_creation=False)
        phases_df_list.append(phase_df)
    phases_cdf = pd.concat(phases_df_list)
    # dislodged_df = phases_cdf.loc[phases_cdf['action'] == -2].copy()
    # utils.replace_dislodged_units(phases_cdf, dislodged_df)
    game_phase_df_list.append(phases_cdf)


In [ ]:
cdf = pd.concat(game_phase_df_list)

### Assert no invalid rows are left

In [ ]:
assert cdf.loc[cdf['coordinator'] == 'RA'].empty

In [ ]:
cdf.loc[cdf['unique_unit_id'].apply(lambda x: x == 1)].head()

### Filter out adjustment and retreat seasons

In [ ]:
spring_fall_phases=(cdf['phase_name'].apply(lambda x:x[0])!='W') & (cdf['phase_name'].apply(lambda x:x[-1])!='R') & (cdf['phase_name'].apply(lambda x:x[-1]) == 'M')
cdf_sf = cdf.loc[spring_fall_phases].copy()
cdf_sf['phase_num']=cdf_sf.phase_name.apply(lambda x: float(x[1:-1]+('.0' if x[0]=='S' else '.5')))

In [ ]:
cdf_sf.head()

### Get the triples' presence based on the filtered dataframe

In [ ]:
game_triple_presence = {}
for idx, game_id in enumerate(unique_games):
    assert type(game_id) is str, (game_id, "is not a string")
    print(idx, game_id)
    game_df = cdf_sf.loc[cdf_sf['game_id'] == game_id]
    
    if game_df.unique_unit_id.nunique() == game_df.unique_unit_id.max():
        triples = utils.get_triples(game_df)
        try:
            emp = utils.get_triples_presence(game_df, triples)
            print(emp, len(triples))
            game_triple_presence[game_id] = triples
        except AssertionError as msg:
            print(msg)
    if idx == 0:
        break

### Save json file if necessary

In [ ]:
    name = "whatever"
    with open(f'{name}.json', 'w') as file:          
        json.dump(game_triple_presence, file, indent=4, sort_keys=True,)

In [ ]:
cdf_sf[cdf_sf["unique_unit_id"].apply(lambda x: x == 1)].head(20)

## Analyze the gathered triples

In [ ]:
path = ""
with open(f'{path}', 'r') as file:
    game_triple_presence = json.load(file)

In [ ]:
list(game_triple_presence.values())[0][0]

In [ ]:
game_id = list(game_triple_presence.keys())[0]

In [ ]:
all_records = []
for row in utils.gen_triple_rows(game_triple_presence):
    all_records.append(row.copy())

In [ ]:
df = pd.DataFrame.from_records(all_records)

In [ ]:
df = df.dropna()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

### Plotting

In [ ]:
samediff_count_list = (df.factor_same.values, df.factor_diff.values)

In [ ]:
a = zip(samediff_count_list[0],samediff_count_list[1])

In [ ]:
fig,ax=plt.subplots()
for data in zip(*filter(None,a)):
    counts,bins=np.histogram(data)
    ax.plot(bins[:-1],counts)

In [ ]:
np.arange(1,2.5,0.5)